In [2]:
import pygame
import random
import time

pygame.init()

# Define the screen dimensions
screen_width = 400
screen_height = 300

# Initialize the game screen
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Human-Playable Game")

# Initialize game state variables
player_size = 10
player_x = screen_width // 2
player_y = screen_height // 2

# Enemy related variables
enemies = []
enemy_spawn_interval = 3.0  # In seconds
enemy_speed = 3  # Pixels moved per enemy update
enemy_lifetime = 5.0  # In seconds
last_enemy_spawn_time = time.time()

# Scoring related variables
score = 0
last_score_time = time.time()

# Powerup related variables
powerup_active = False
powerup_spawn_interval = 10.0  # In seconds
last_powerup_spawn_time = time.time()
powerup_x = 0
powerup_y = 0

# Font for score display
font = pygame.font.Font(None, 36)

clock = pygame.time.Clock()

def spawn_enemy():
    enemy_size = random.uniform(0.5, 1.5) * player_size
    enemy_x = random.randint(0, screen_width - 1)
    enemy_y = random.randint(0, screen_height - 1)
    enemy_spawn_time = time.time()
    enemies.append((enemy_x, enemy_y, enemy_size, enemy_spawn_time))

def move_enemies():
    global enemy_speed, powerup_active
    for i, (enemy_x, enemy_y, enemy_size, enemy_spawn_time) in enumerate(enemies):
        speed = enemy_speed

        # Enemies move randomly
        direction = random.choice(['up', 'down', 'left', 'right'])
        if direction == 'up':
            enemies[i] = (enemy_x, max(0, enemy_y - speed), enemy_size, enemy_spawn_time)
        elif direction == 'down':
            enemies[i] = (enemy_x, min(screen_height - 1, enemy_y + speed), enemy_size, enemy_spawn_time)
        elif direction == 'left':
            enemies[i] = (max(0, enemy_x - speed), enemy_y, enemy_size, enemy_spawn_time)
        elif direction == 'right':
            enemies[i] = (min(screen_width - 1, enemy_x + speed), enemy_y, enemy_size, enemy_spawn_time)

def spawn_powerup():
    global powerup_active, powerup_x, powerup_y
    powerup_x = random.randint(0, screen_width - 1)
    powerup_y = random.randint(0, screen_height - 1)
    powerup_active = True

def remove_old_enemies():
    global enemies, enemy_lifetime
    current_time = time.time()
    enemies = [(x, y, size, spawn_time) for x, y, size, spawn_time in enemies
                if current_time - spawn_time <= enemy_lifetime]

def draw_objects():
    screen.fill((0, 0, 0))
    pygame.draw.rect(screen, (255, 255, 255), (player_x, player_y, player_size, player_size))
    for enemy_x, enemy_y, enemy_size, _ in enemies:
        pygame.draw.rect(screen, (255, 0, 0), (enemy_x, enemy_y, enemy_size, enemy_size))
    if powerup_active:
        pygame.draw.rect(screen, (0, 255, 0), (powerup_x, powerup_y, 10, 10))
    
    # Display the score
    score_display = font.render(f"Score: {score}", True, (255, 255, 255))
    screen.blit(score_display, (10, 10))
    
    pygame.display.flip()
    
def is_colliding(x1, y1, size1, x2, y2, size2, buffer=5):
    return (x1 + size1 + buffer > x2 and x1 < x2 + size2 + buffer and
            y1 + size1 + buffer > y2 and y1 < y2 + size2 + buffer)

def pick_up_powerup():
    global powerup_active, score, enemy_speed, enemy_spawn_interval

    if powerup_active and is_colliding(player_x, player_y, player_size, powerup_x, powerup_y, 10):
        score += 100
        powerup_active = False

        # Increase enemy speed and decrease enemy spawn interval
        enemy_speed += 5
        enemy_spawn_interval = max(0.5, enemy_spawn_interval - 0.5)

running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_UP]:
        player_y -= 5
    if keys[pygame.K_DOWN]:
        player_y += 5
    if keys[pygame.K_LEFT]:
        player_x -= 5
    if keys[pygame.K_RIGHT]:
        player_x += 5

    # Clip player position to screen boundaries
    player_x = max(0, min(player_x, screen_width - 1))
    player_y = max(0, min(player_y, screen_height - 1))

    # Update the score
    current_time = time.time()
    if current_time - last_score_time > 1.0:
        score += 1
        last_score_time = current_time

    # Spawn new enemy periodically
    if current_time - last_enemy_spawn_time > enemy_spawn_interval:
        spawn_enemy()
        last_enemy_spawn_time = current_time

    # Spawn powerup periodically
    if not powerup_active and current_time - last_powerup_spawn_time > powerup_spawn_interval:
        spawn_powerup()
        last_powerup_spawn_time = current_time

    # Move enemies
    move_enemies()

    # Check if player picks up the powerup
    pick_up_powerup()

    # Remove old enemies
    remove_old_enemies()

    # Check for collisions between the player and enemies
    for enemy_x, enemy_y, enemy_size, _ in enemies:
        if is_colliding(player_x, player_y, player_size, enemy_x, enemy_y, enemy_size):
            # Game over if player collides with an enemy
            running = False

    draw_objects()
    clock.tick(30)

pygame.quit()
